In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random

In [3]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random

DS = Path("/Users/hariomnarang/Desktop/personal/roads/datasets/")
EXP_BASE = DS / "T004-taco-crops"
OUTS = EXP_BASE / "synth"
LOG_BASE = EXP_BASE / "log"
TACO_BASE_DIR = Path("/Users/hariomnarang/Desktop/personal/TACO/data/")
ANN_FILE = TACO_BASE_DIR / "annotations.json"


LOG_BASE.mkdir(parents=True, exist_ok=True)
DS.exists(), TACO_BASE_DIR.exists(), ANN_FILE.exists()

(True, True, True)

# Generate Data

In [4]:
# generate data first

from mtrain.smallnet import extract_all

In [5]:
extract_all(TACO_BASE_DIR, ANN_FILE, OUTS, 50, 10)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


Generating samples: 100%|██████████| 10/10 [00:00<00:00, 58.38iteration/s]


# Training

In [4]:
from fastai.vision.all import ImageDataLoaders, get_image_files


def label_func(path):
    return Path(path).parent.name

dls = ImageDataLoaders.from_name_func(
    LOG_BASE, get_image_files(OUTS), valid_pct=0.2, seed=42, label_func=label_func
)

In [5]:
from fastai.vision.all import vision_learner, mobilenet_v3_small, CrossEntropyLossFlat, mobilenet_v2, resnet18
# from fastai.vision.all import *

In [6]:
learner = vision_learner(dls, mobilenet_v3_small, normalize=True, pretrained=True)

/Users/hariomnarang/Desktop/personal/roads/mtrain/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hariomnarang/Desktop/personal/roads/mtrain/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# with learner.no_bar(), learner.no_logging():
learner.fine_tune(1)

AttributeError: Exception occured in `ProgressCallback` when calling event `before_fit`:
	'NBMasterBar' object has no attribute 'out'